# Trabalho de PDI

Importando bibliotecas

In [1]:
from PIL import Image
from PIL import ImageFilter
import imageio as im
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import data
from skimage import filters
from skimage import exposure
from skimage.feature import hog
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

Carregando imagens do computador

In [2]:
def loadImages(path):
    # return array of images
    path = os.getcwd() + '/' + path
    imagesList = os.listdir(path)
    loadedImages = []
    for image in imagesList:
        #img = im.imread(os.path.join(path, image))
        img = Image.open(os.path.join(path, image)).convert('L')
        loadedImages.append(img)

    return loadedImages

correctExamples = loadImages("/images/correctexamples/")
incorrectExamples = loadImages("/images/incorrectexamples/")
originalExamples = loadImages("/images/originals/")


In [3]:
def plot(img):
    plt.figure(figsize=(20,10))
    imgplot = plt.imshow(img)

    plt.show()

In [4]:
def preprocess(imgs):
    imgs = [img.filter(ImageFilter.MedianFilter()) for img in imgs]
    return imgs
corrects = preprocess(correctExamples)
incorrects = preprocess(incorrectExamples)
originals = preprocess(originalExamples)


In [5]:
def toNP(img):
    return np.asarray(img, dtype=np.uint8)

Gerando histograma de gradiente para todas as imagens

In [6]:
def getBins(fd, num_bins = 20, max_val = 1, min_val = 0):
    total = max_val - min_val
    return [min_val + i*total/num_bins for i in range(num_bins + 1)]
def getHistogram(arr):
    hist, _ = np.histogram(arr , bins = getBins(arr))
    hist = hist / hist.max()
    return hist
def extractHistogram(img):
    fd, _ = hog(img, orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(1, 1), visualize=True, block_norm='L2-Hys')
    return getHistogram(fd)
correctsHistogram = [extractHistogram(img) for img in corrects]
incorrectsHistogram = [extractHistogram(img) for img in incorrects]
originalsHistogram = [extractHistogram(img) for img in originals]

In [7]:
def plotHistogram(hist):
    plt.plot(hist)
    plt.show()
def compareHistograms(h1, h2):
    x = [[i, h1[i]]for i in range(len(h1))]
    y = [[i, h2[i]]for i in range(len(h2))]
    dist, _ = fastdtw(x, y, dist=euclidean)
    return dist
def compareAllBase(h1, base):
    dist = 1000
    for h2 in base:
        dist = min(dist, compareHistograms(h1, h2))
    return dist

shouldBeCorrect = [compareAllBase(hist, originalsHistogram) for hist in correctsHistogram]
shouldBeIncorrect = [compareAllBase(hist, originalsHistogram) for hist in incorrectsHistogram]

threshold = 1.1
n_correct_positive = len([x for x in shouldBeCorrect if x < threshold])
true_positive = n_correct_positive/len(corrects)
n_incorrect_positive = len([x for x in shouldBeIncorrect if x > threshold])
true_negative = n_incorrect_positive/len(incorrects)

print("Verdadeiros Positivos: {}".format(true_positive))
print("Verdadeiros Negativos: {}".format(true_negative))


Verdadeiros Positivos: 0.6923076923076923
Verdadeiros Negativos: 0.6


O algoritmo classificou corretamente 69% das imagens com correspondência, e 60% das imagens sem correspondência